In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import argparse
import os

from MethylCDM.utils.utils import init_environment, load_config, resolve_path
from MethylCDM.data.load_methylation import download_methylation, merge_cohort
from MethylCDM.preprocessing.process_methylation import process_methylation
from MethylCDM.constants import (
    RAW_METHYLATION_DIR, 
    PROCESSED_METHYLATION_DIR,
    METADATA_METHYLATION_DIR
)

# Mimic command-line arguments
args = {
    "project": "TCGA-BRCA",
    "config_pipeline": "pipeline.yaml",
    "config_preproc": "methylation_preproc.yaml",
    "verbose": True
}

# Load the relevant configuration files 
pipeline_cfg = load_config(args['config_pipeline'])
preproc_cfg = load_config(args['config_preproc'])

# Initialize the environment for reproducible analysis
init_environment(pipeline_cfg)

# Mimic function parameters
project = args['project']
config = preproc_cfg

In [3]:
from MethylCDM.utils.utils import load_annotation, load_beta_file
from MethylCDM.constants import (
    CONFIG_DIR,
    ANNOTATION_27K,
    ANNOTATION_450K,
    ANNOTATION_EPIC
)

from MethylCDM.preprocessing.process_methylation import (
    sample_qc,
    probe_qc,
    impute_missing,
    aggregate_genes,
    clip_beta_values
)

metadata_dir = preproc_cfg.get('download', {}).get('metadata_dir', '')
metadata_dir = resolve_path(metadata_dir, METADATA_METHYLATION_DIR)
project_metadata = os.path.join(metadata_dir, args['project'],
                                f"{args['project']}_metadata.csv")
metadata = pd.read_csv(project_metadata)

from pathlib import Path
import pandas as pd
import numpy as np
import anndata as ad
from collections import defaultdict
import os
from MethylCDM.utils.utils import (
    resolve_path,
    load_beta_file,
    load_annotation,
    load_cpg_matrix
)
from MethylCDM.constants import RAW_METHYLATION_DIR
from MethylCDM.preprocessing.process_methylation import (
    process_array_methylation
)

# Resolve the project's raw data directory
raw_data_dir = config.get('download', {}).get('raw_data_dir', '')
raw_data_dir = resolve_path(raw_data_dir, RAW_METHYLATION_DIR)
project_data_dir = os.path.join(raw_data_dir, f"{project}")

# Verify the raw data exists and is not empty
if not os.path.isdir(project_data_dir):
    raise FileNotFoundError(f"Raw data directory was not found at "
                            f"{project_data_dir}.")
if not os.listdir(project_data_dir):
    raise FileNotFoundError(f"Raw data directory was empty at "
                            f"{project_data_dir}.")

# Identify and load all nested beta value .txt files
beta_files = [
    f for f in Path(project_data_dir).glob("*.level3betas.parquet")
]

# Keep the first occurrence of duplicates in the metadata (redundant)
metadata = metadata.set_index('file_name')
metadata = metadata[~metadata.index.duplicated(keep = 'first')]


In [4]:
# Identify the highest coverage array type
manifests = metadata['platform'].unique()
annotation, array_type = load_annotation(manifests)

# Normalize the metadata extensions
metadata.index = metadata.index.str.removesuffix(".txt")
valid_stems = set(metadata[metadata['platform'] == array_type].index)

# Fetch samples that align with the given array type
beta_files = [f for f in beta_files if f.stem in valid_stems]

In [7]:
# Preprocess the beta values into a gene-level matrix
cpg_matrix = load_cpg_matrix(beta_files)

In [ ]:
gene_matrix = process_array_methylation(cpg_matrix, annotation, config)
gene_matrix = gene_matrix.sort_index()

In [22]:
# Filter the metadata for surviving samples
metadata = metadata.loc[gene_matrix.columns]
metadata = metadata.sort_index()

In [25]:
# Initialize the gene matrix as an AnnData object
adata = ad.AnnData(X = gene_matrix.T)
adata.obs = metadata

In [32]:
gene_matrix = adata

In [36]:
proc_data_dir = (preproc_cfg.get('preprocess', {})
                        .get('processed_data_dir', ''))
proc_data_dir = resolve_path(proc_data_dir, PROCESSED_METHYLATION_DIR)
proc_file = os.path.join(proc_data_dir, args['project'],
                        f"{args['project']}_gene_matrix.h5ad")

gene_matrix.write_h5ad(proc_file, compression = "gzip", compression_opts = 4)